In [3]:
import pandas as pd

urla1 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv"

url1a2 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_dogs.csv"

urla3 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv"

urla4 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla5 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla6 ="https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_item_associations.csv"

analysis1  = pd.read_csv(urla1)
analysis2  = pd.read_csv(url1a2)
analysis3  = pd.read_csv(urla3)
analysis4  = pd.read_csv(urla4)
item_add = pd.read_csv(urla5)
item_item = pd.read_csv(urla6)



In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report




url_menu_items = (
   "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv"

)

df = pd.read_csv(url_menu_items)

print("Columns in dataset:")
print(df.columns.tolist())


#  BASIC CLEANING


# Required columns for menu engineering
required_cols = ["price", "purchases"]

df = df.dropna(subset=required_cols)

# Ensure numeric
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["purchases"] = pd.to_numeric(df["purchases"], errors="coerce")

df = df.dropna(subset=required_cols)


#  FEATURE ENGINEERING (BUSINESS-ALIGNED)


# Revenue
df["revenue"] = df["price"] * df["purchases"]

# Estimated cost (40% food cost assumption)
df["estimated_cost"] = df["price"] * 0.40

# Contribution margin
df["margin"] = df["price"] - df["estimated_cost"]


# CREATE MENU ENGINEERING LABELS (GROUND TRUTH)


popularity_threshold = df["purchases"].median()
profit_threshold = df["margin"].median()

def menu_engineering_label(row):
    if row["purchases"] >= popularity_threshold and row["margin"] >= profit_threshold:
        return "Star"
    elif row["purchases"] >= popularity_threshold and row["margin"] < profit_threshold:
        return "Plowhorse"
    elif row["purchases"] < popularity_threshold and row["margin"] >= profit_threshold:
        return "Puzzle"
    else:
        return "Dog"

df["menu_label"] = df.apply(menu_engineering_label, axis=1)

print("\nMenu category distribution:")
print(df["menu_label"].value_counts())

# PREPARE DATA FOR CLASSIFICATION MODEL


features = ["price", "purchases", "margin"]
X = df[features]
y = df["menu_label"]

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

# TRAIN CLASSIFICATION MODEL

model = DecisionTreeClassifier(
    max_depth=4,
    random_state=42
)

model.fit(X_train, y_train)

#  EVALUATION
y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=label_encoder.classes_
))




Columns in dataset:
['id', 'title', 'description', 'price', 'purchases', 'desc_length']

Menu category distribution:
menu_label
Star         4677
Puzzle       4674
Plowhorse    4650
Dog          4649
Name: count, dtype: int64

Classification Report:
              precision    recall  f1-score   support

         Dog       1.00      1.00      1.00       930
   Plowhorse       1.00      1.00      1.00       930
      Puzzle       1.00      1.00      1.00       935
        Star       1.00      1.00      1.00       935

    accuracy                           1.00      3730
   macro avg       1.00      1.00      1.00      3730
weighted avg       1.00      1.00      1.00      3730



In [4]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.preprocessing import LabelEncoder

urla1 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv"

url1a2 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_dogs.csv"

urla3 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv"

urla4 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla5 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla6 ="https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_item_associations.csv"

df_main = pd.read_csv(urla3)
df_item_assoc = pd.read_csv(urla6)
df_addon_assoc = pd.read_csv(urla5)
df_desc = pd.read_csv(urla1)
df_whatif = pd.read_csv(urla4)


df_main = df_main.merge(df_desc[['id', 'desc_length']], left_on='item_id', right_on='id', how='left').fillna(0)

# 3. TRAIN DECISION TREE (The "Brain")
# Features: Price, Purchases, and the Description Length mentioned in your plan
features = ['price', 'purchases', 'desc_length']
X = df_main[features]
y = df_main['category']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

# We use a depth of 3 to make the rules clear for business stakeholders
dt_model = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_model.fit(X, y_encoded)

# 4. INTEGRATED RECOMMENDATION SYSTEM
def build_recommendation_engine(item_name):
    # Determine the item's category from the model
    row = df_main[df_main['item_name'] == item_name]
    if row.empty: return "Item not found"

    category = row['category'].values[0]

    # BUSINESS LOGIC
    recommendations = []

    # Strategy A: If it's a PUZZLE -> Find a STAR to bundle with
    if category == 'Puzzle':
        # Search associations for this item
        assoc = df_item_assoc[df_item_assoc['item_a'] == item_name]
        # Filter for only those associated items that are STARS
        stars = df_main[df_main['category'] == 'Star']['item_name'].tolist()
        bundles = assoc[assoc['item_b'].isin(stars)]
        recommendations = bundles[['item_b', 'confidence']].rename(columns={'item_b': 'Bundle Partner (Star)'})

    # Strategy B: If it's a PLOWHORSE -> Show What-If Simulation
    elif category == 'Plowhorse':
        sim = df_whatif[df_whatif['item_name'] == item_name]
        recommendations = sim[['profit', 'profit_price_up']].rename(columns={'profit_price_up': 'Projected Profit (Price +5%)'})

    # Strategy C: If it's a STAR -> Suggest high-confidence Add-ons (Upsell Map)
    elif category == 'Star':
        # Use the Add-on associations for top-tier upselling
        addons = df_addon_assoc.sort_values('lift', ascending=False).head(3)
        recommendations = addons[['consequent', 'confidence']].rename(columns={'consequent': 'Top Add-on Upsell'})

    return category, recommendations


# Example: Get strategy for a known Star
cat, recs = build_recommendation_engine('The Classic')
print(f"Strategy for '{cat}':")
print(recs)

# Export the classified items for your final report
df_main.to_csv('final_integrated_menu_analysis.csv', index=False)

Strategy for 'Star':
    Top Add-on Upsell  confidence
384             Tomat    0.787288
207         Oksefilet    0.876357
24              Salat    0.980777
